## Cross-Lingual Voice Clone Demo

In [1]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



### Initialization

In [32]:
ckpt_converter = 'checkpoints_v2/converter'
device="cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())
print(torch.backends.cuda.is_built())
print(torch.cuda.is_available())

output_dir = 'outputs'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

cpu
True
True
False
False


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


In this demo, we will use OpenAI TTS as the base speaker to produce multi-lingual speech audio. The users can flexibly change the base speaker according to their own needs. Please create a file named `.env` and place OpenAI key as `OPENAI_API_KEY=xxx`. We have also provided a Chinese base speaker model (see `demo_part1.ipynb`).

In [3]:
from openai import OpenAI
from dotenv import load_dotenv

# Please create a file named .env and place your
# OpenAI key as OPENAI_API_KEY=xxx
load_dotenv() 

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

response = client.audio.speech.create(
    model="tts-1",
    voice="nova",
    input="This audio will be used to extract the base speaker tone color embedding. " + \
        "Typically a very short audio should be sufficient, but increasing the audio " + \
        "length will also improve the output audio quality."
)

response.stream_to_file(f"{output_dir}/openai_source_output.mp3")

ModuleNotFoundError: No module named 'openai'

# copy from demo_part1.ipynb 
from openvoice.api import BaseSpeakerTTS, ToneColorConverter

In [21]:
from openvoice.api import BaseSpeakerTTS

ckpt_base = 'checkpoints/base_speakers/ZH'
base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

source_se = torch.load(f'{ckpt_base}/zh_default_se.pth').to(device)

# ckpt_base2 = 'checkpoints/base_speakers/EN'
# target_se = torch.load(f'{ckpt_base2}/en_style_se.pth').to(device)
target_se = torch.load(f'checkpoints_v2/base_speakers/ses/en-india.pth').to(device)
save_path = f'{output_dir}/output_english_india.wav'

# Run the base speaker tts
# text = "今天天气真好，我们一起出去吃饭吧.今天是星期四，还有两天放假了。"
text = "MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps."
src_path = f'{output_dir}/tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints/base_speakers/ZH/checkpoint.pth'
missing/unexpected keys: [] []
 > Text splitted to sentences.
MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps.
 > ===========================
maɪ ʃɛɫ ɪz ə dɪˈsɛntɹəˌlaɪzd ənd ˌkɑmpɹiˈhɛnsɪv ˈplætˌfɔɹm fəɹ dɪˈskəvəɹɪŋ, kɹiˈeɪtɪŋ, ənd ˈsteɪkɪŋ ai-native* æps.
 length:115
 length:115


In [22]:
from openvoice.api import BaseSpeakerTTS

ckpt_base = 'checkpoints/base_speakers/EN'
base_speaker_tts = BaseSpeakerTTS(f'{ckpt_base}/config.json', device=device)
base_speaker_tts.load_ckpt(f'{ckpt_base}/checkpoint.pth')

source_se = torch.load(f'{ckpt_base}/en_style_se.pth').to(device)
target_se = torch.load(f'checkpoints_v2/base_speakers/ses/en-india.pth').to(device)
save_path = f'{output_dir}/output_english_india.wav'

# Run the base speaker tts
text = "MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps."
src_path = f'{output_dir}/en_tmp.wav'
base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
 > Text splitted to sentences.
MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps.
 > ===========================
maɪ ʃɛɫ ɪz ə dɪˈsɛntɹəˌlaɪzd ənd ˌkɑmpɹiˈhɛnsɪv ˈplætˌfɔɹm fəɹ dɪˈskəvəɹɪŋ, kɹiˈeɪtɪŋ, ənd ˈsteɪkɪŋ ai-native* æps.
 length:115
 length:115


In [27]:
source_se = torch.load(f'checkpoints_v2/base_speakers/ses/en-us.pth').to(device)
src_path = f'resources/demo_speaker0.mp3'

reference_speaker = 'resources/example_reference.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)
save_path = f'{output_dir}/output_english_man.mp3'

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

OpenVoice version: v2


FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

是否可修改TTS的配置呢
ckpt_base2 = 'checkpoints/base_speakers/EN'


In [17]:
ckpt_base2 = 'checkpoints_v2/base_speakers/EN'
base_speaker_tts2 = BaseSpeakerTTS(f'{ckpt_base2}/config.json', device=device)
base_speaker_tts2.load_ckpt(f'{ckpt_base2}/checkpoint.pth')
text2 = "Demo Usage. Please see demo_part3.ipynb for example usage of OpenVoice V2. Now it natively supports English, Spanish, French, Chinese, Japanese and Korean."
output_texg2 = f'{output_dir}/text2s.wav'
base_speaker_tts2.tts(text2, output_texg2, speaker='default', language='English', speed=1.0)


Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
 > Text splitted to sentences.
Demo Usage. Please see demo_part3. ipynb for example usage of OpenVoice V2.
Now it natively supports English, Spanish, French, Chinese, Japanese and Korean.
 > ===========================
ˈdɛmoʊ ˈjusɪdʒ. pliz si demo_paɹtthɹee*. ipynb* fəɹ ɪgˈzæmpəɫ ˈjusɪdʒ əv ˈoʊpən vɔɪs vtwo*.
 length:91
 length:91
naʊ ɪt nativeɫy* səˈpɔɹts ˈɪŋlɪʃ, ˈspænɪʃ, fɹɛntʃ, tʃaɪˈniz, ˌdʒæpəˈniz ənd ˌkɔˈɹiən.
 length:85
 length:85


### Obtain Tone Color Embedding

The `source_se` is the tone color embedding of the base speaker. 
It is an average for multiple sentences with multiple emotions
of the base speaker. We directly provide the result here but
the readers feel free to extract `source_se` by themselves.

base_speaker = f"{output_dir}/openai_source_output.mp3"

In [15]:
base_speaker = f"{output_dir}/output_en.wav"
source_se, audio_name = se_extractor.get_se(base_speaker, tone_color_converter, vad=True)

reference_speaker = 'resources/example_reference.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)

OpenVoice version: v2


FileNotFoundError: [Errno 2] No such file or directory: 'ffmpeg'

### Inference

In [20]:
# Run the base speaker tts
text = [
    "MyShell is a decentralized and comprehensive platform for discovering, creating, and staking AI-native apps.",
    "MyShell es una plataforma descentralizada y completa para descubrir, crear y apostar por aplicaciones nativas de IA.",
    "MyShell est une plateforme décentralisée et complète pour découvrir, créer et miser sur des applications natives d'IA.",
    "MyShell ist eine dezentralisierte und umfassende Plattform zum Entdecken, Erstellen und Staken von KI-nativen Apps.",
    "MyShell è una piattaforma decentralizzata e completa per scoprire, creare e scommettere su app native di intelligenza artificiale.",
    "MyShellは、AIネイティブアプリの発見、作成、およびステーキングのための分散型かつ包括的なプラットフォームです。",
    "MyShell — это децентрализованная и всеобъемлющая платформа для обнаружения, создания и стейкинга AI-ориентированных приложений.",
    "MyShell هي منصة لامركزية وشاملة لاكتشاف وإنشاء ورهان تطبيقات الذكاء الاصطناعي الأصلية.",
    "MyShell是一个去中心化且全面的平台，用于发现、创建和投资AI原生应用程序。",
    "MyShell एक विकेंद्रीकृत और व्यापक मंच है, जो AI-मूल ऐप्स की खोज, सृजन और स्टेकिंग के लिए है।",
    "MyShell é uma plataforma descentralizada e abrangente para descobrir, criar e apostar em aplicativos nativos de IA."
]
src_path = f'{output_dir}/tmp.wav'

for i, t in enumerate(text):

    response = client.audio.speech.create(
        model="tts-1",
        voice="nova",
        input=t,
    )

    response.stream_to_file(src_path)

    save_path = f'{output_dir}/output_crosslingual_{i}.wav'

    # Run the tone color converter
    encode_message = "@MyShell"
    tone_color_converter.convert(
        audio_src_path=src_path, 
        src_se=source_se, 
        tgt_se=target_se, 
        output_path=save_path,
        message=encode_message)

NameError: name 'client' is not defined